<div align="center">

# Play Ground Series S3 E26

## 💉 Predicting outcomes of patients with cirrhosis 💉

### Welcome to my kaggle notebook! For this Episode of the Series, is to use a multi-class approach to predict the the outcomes of patients with cirrhosis.

</div>

## Goal: For this Episode of the Series, is to use a multi-class approach to predict the the outcomes of patients with cirrhosis

In [9]:
# Standard library imports
import pandas as pd
import numpy as np
import seaborn as sns
import ydata_profiling as pp

# Plotting libraries
import matplotlib.pyplot as plt
import ipywidgets as widgets
#import plotly.express as px

# ML imports

In [10]:
df_train = pd.read_csv(r"..\data\train.csv")
df_test = pd.read_csv(r"..\data\test.csv")

In [ ]:
pp.ProfileReport(df_train)

In [8]:
def analyze_dataframe(df):
    """
    Analyze a pandas DataFrame and provide a summary of its characteristics.

    Parameters:
    df (pandas.DataFrame): The input DataFrame to analyze.

    Returns:
    None
    """
    print("DataFrame Information:")
    print("______________________")
    display(df.info(verbose=True, show_counts=True))
    print("\n")
    
    print("DataFrame Head:")
    print("______________________")
    display(df.head())
    print("\n")

    print("DataFrame Tail:")
    print("______________________")
    display(df.tail())
    print("\n")

    print("DataFrame Description:")
    print("______________________")
    display(df.describe().T)
    print("\n")

    print("Number of Null Values:")
    print("______________________")
    display(df.isnull().sum())
    print("\n")

    print("Number of Duplicated Rows:")
    print("______________________")
    display(df.duplicated().sum())
    print("\n")

    print("Number of Unique Values:")
    print("______________________")
    display(df.nunique())
    print("\n")

    print("DataFrame Shape:")
    print("______________________")
    print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

analyze_dataframe(df_train)

DataFrame Information:
______________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7905 entries, 0 to 7904
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             7905 non-null   int64  
 1   N_Days         7905 non-null   int64  
 2   Drug           7905 non-null   object 
 3   Age            7905 non-null   int64  
 4   Sex            7905 non-null   object 
 5   Ascites        7905 non-null   object 
 6   Hepatomegaly   7905 non-null   object 
 7   Spiders        7905 non-null   object 
 8   Edema          7905 non-null   object 
 9   Bilirubin      7905 non-null   float64
 10  Cholesterol    7905 non-null   float64
 11  Albumin        7905 non-null   float64
 12  Copper         7905 non-null   float64
 13  Alk_Phos       7905 non-null   float64
 14  SGOT           7905 non-null   float64
 15  Tryglicerides  7905 non-null   float64
 16  Platelets      7905 non-null   float64
 17  Prothr

None



DataFrame Head:
______________________


,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C




DataFrame Tail:
______________________


...



Number of Duplicated Rows:
______________________


0



Number of Unique Values:
______________________


id               7905
N_Days            461
Drug                2
Age               391
Sex                 2
Ascites             2
Hepatomegaly        2
Spiders             2
Edema               3
Bilirubin         111
Cholesterol       226
Albumin           160
Copper            171
Alk_Phos          364
SGOT              206
Tryglicerides     154
Platelets         227
Prothrombin        49
Stage               4
Status              3
dtype: int64



DataFrame Shape:
______________________
Rows: 7905, Columns: 20


## Data points interpretation
___
1. **Ascites**: Ascites is a medical term for an abnormal fluid buildup in the abdominal cavity, specifically in the peritoneal space. This syndrome is frequently caused by underlying health conditions such as liver disease, cirrhosis, heart failure, cancer, or infections.

2. **Hepatomegaly**: Hepatomegaly refers to an enlargement of the liver. It is a non-specific medical sign, having many causes, which can broadly be broken down into infection, hepatic tumors, and metabolic disorder.

3. **Spiders**: In medical terms, 'spiders' could refer to spider angiomas (also known as spider nevi), which are common skin lesions consisting of central arterioles surrounded by many smaller vessels due to high estrogen levels and may occur in any condition with high estrogen states such as cirrhosis.

4. **Edema**: Edema is swelling caused by fluid trapped in your body’s tissues, most often in your feet, ankles, and legs. Your healthcare provider will test your edema by pressing their finger into the swollen area (pitting) to identify how much fluid is in your tissues (grade).

5. **Bilirubin**: Bilirubin is a reddish-yellow water-insoluble pigment that is formed by the breakdown of heme, is excreted in a water-soluble form by liver cells into bile, and occurs in blood and urine especially in diseased states.

6. **Cholesterol**: Cholesterol is a waxy, fat-like substance that's found in all the cells in your body. Your body needs some cholesterol to make hormones, vitamin D, and substances that help you digest foods.

7. **Albumin**: Albumin is a type of protein that is found in your blood. It's produced by your liver and serves several important functions in the body. One of its main roles is to help maintain the right amount of water in your blood and tissues.

8. **Copper**: Copper, an essential mineral, is naturally present in some foods and is available as a dietary supplement. It is a cofactor for several enzymes (known as cuproenzymes) involved in energy production, iron metabolism, 
neuropeptide activation, connective tissue synthesis, and neurotransmitter synthesis.

9. **Alk_Phos**: Alkaline phosphatase (ALP) is an enzyme that’s found throughout your body. An enzyme is a type of protein in a cell that acts as a catalyst and allows certain bodily processes to happen.

10. **SGOT**: Serum glutamic oxaloacetic transaminase (SGOT), also known as aspartate aminotransferase (AST), is an enzyme that is normally present in liver and heart cells. SGOT is released into blood when the liver or heart is damaged.

## Possible interactions of the data points
___
1. **Bilirubin and Albumin**: These are both liver function tests. Bilirubin levels can increase due to liver dysfunction, while albumin levels can decrease. Their interaction might be indicative of the severity of liver disease.

2. **Ascites and Edema with Albumin**: Ascites and edema are clinical signs of advanced liver disease and can be influenced by the level of albumin, as it plays a crucial role in maintaining oncotic pressure in the blood vessels.

3. **Alkaline Phosphatase (Alk_Phos) and Bilirubin**: Both are markers of liver function. Elevated levels can indicate cholestasis or blockage of bile flow, often seen in liver diseases.

4. **SGOT and Bilirubin**: SGOT is an enzyme that can be elevated in liver damage. Together with bilirubin, these levels can indicate the extent of liver injury.

5. **Platelets and Prothrombin**: Both are involved in blood clotting. Liver disease can lead to thrombocytopenia (low platelet count) and prolonged prothrombin time, reflecting impaired liver synthesis function.

6. **Cholesterol and Triglycerides**: These are both lipids, and their interaction can be relevant in understanding the overall lipid profile, which is important in the context of cardiovascular risk and metabolic health.

7. **Drug (D-penicillamine) with Liver Function Tests**: The interaction between the use of D-penicillamine and liver function tests like Bilirubin, Albumin, Alk Phos might be insightful, especially in patients with Wilson’s Disease or liver involvement in Rheumatoid Arthritis.

8. **Age with Various Biomarkers**: Age might modulate the relationship between various biomarkers (like Bilirubin, Albumin, SGOT) and clinical outcomes.



In [ ]:
def prelim_eda_histplot(df):
    """
    Create a histogram plot of each column in a pandas DataFrame.

    Parameters:
    df (pandas.DataFrame): The input DataFrame to analyze.

    Returns:
    None
    """
    columns_to_plot = df.drop(['id','Drug','Sex','Ascites','Hepatomegaly','Spiders','Edema','Stage','Status'], axis=1)      


    num_columns = len(columns_to_plot.columns)
    num_rows = (num_columns - 1) // 4 + 1  # Calculate the number of rows based on the number of columns
    num_cols = min(num_columns, 4)  # Set the number of columns to 4 or the number of columns in the DataFrame, whichever is smaller
    fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(20, 12))  # Adjust the figsize as needed

    # Loop over selected columns and create histogram plots in separate subplots
    for i, column in enumerate(columns_to_plot):
        row = i // num_cols
        col = i % num_cols
        sns.histplot(data=df, x=column, ax=axes[row, col], kde=True, bins=20)
        axes[row, col].set_title(f'{column}', fontsize=14)
        axes[row, col].set_aspect('auto')

    # Remove empty subplots (if any)
    for i in range(num_columns, num_rows * num_cols):
        fig.delaxes(axes.flatten()[i])

    # Adjust layout
    plt.tight_layout()
    plt.show()

prelim_eda_histplot(df_train)

In [ ]:

def create_boxplots(df, exclude_columns):
    """
    Create a series of boxplot subplots for the given DataFrame, excluding specified columns.

    Parameters:
    df (DataFrame): The DataFrame containing the data.
    exclude_columns (list): List of column names to exclude from plotting.

    Returns:
    matplotlib.figure.Figure: The figure object containing the subplots.
    """
    # Dropping the specified columns
    df_to_plot = df.drop(exclude_columns, axis=1)
    num_cols = len(df_to_plot.columns)
    
    # Determining the layout of the subplot grid
    cols_per_row = 3
    rows = (num_cols + cols_per_row - 1) // cols_per_row

    # Creating the subplots
    fig, axes = plt.subplots(rows, cols_per_row, figsize=(15, 5 * rows))
    axes = axes.flatten()  # Flatten in case of a single row

    for i, col in enumerate(df_to_plot.columns):
        df_to_plot.boxplot(column=col, ax=axes[i])
        axes[i].set_title(col)

    # Hide unused subplots
    for j in range(i + 1, rows * cols_per_row):
        fig.delaxes(axes[j])

    plt.tight_layout()
    return fig

# Columns to exclude from the plot
exclude_columns = ['id', 'Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Stage', 'Status']

# Creating the boxplot subplots
fig = create_boxplots(df_train, exclude_columns)
plt.show()


In [ ]:
def create_heatmap(df, figsize=(10, 8), annot=True, cmap='coolwarm',fmt='.1f'):
    """
    Create a heatmap for the correlation matrix of the numeric columns in the given DataFrame.

    Parameters:
    df (DataFrame): The DataFrame containing the data.
    figsize (tuple): Size of the heatmap.
    annot (bool): If True, write the data value in each cell.
    cmap (str): Colormap used for the heatmap.

    Returns:
    matplotlib.figure.Figure: The figure object containing the heatmap.
    """
    plt.figure(figsize=figsize)

    # Select only numeric columns for correlation matrix
    numeric_df = df.select_dtypes(include=[np.number])
    correlation_matrix = numeric_df.corr()

    sns.heatmap(correlation_matrix, annot=annot, cmap=cmap, fmt=fmt)
    plt.title('Heatmap of Correlation Matrix')
    plt.show()

# Example usage
create_heatmap(df_train)

# Feature Engineering: Converting Age from days to years and creating interaction features  and liver health indicators
___

In [11]:
def convert_age_and_create_interactions(df):
    """
    Convert 'Age' from days to years and create interaction features.

    Parameters:
    df (pandas.DataFrame): The input DataFrame.

    Returns:
    pandas.DataFrame: The modified DataFrame.
    """
    # Feature Engineering: Categorical Variable Interactions and Grouped Categorical Variables

    # Creating interaction features for categorical variables
    df['Ascites_Edema'] = df['Ascites'] + "_" + df['Edema']
    df['Ascites_Spiders'] = df['Ascites'] + "_" + df['Spiders']
    df['Hepatomegaly_Spiders'] = df['Hepatomegaly'] + "_" + df['Spiders']

    # Creating grouped categorical variables based on health
    # Grouping liver health indicators: Ascites, Hepatomegaly, Spiders
    # Counting the number of 'Y' (Yes) responses in these columns
    df['Liver_Health_Indicators'] = df[['Ascites', 'Hepatomegaly', 'Spiders']].apply(lambda x: (x == 'Y').sum(), axis=1)

    df['Age_years'] = df['Age'] / 365.25
    df['Bilirubin_Albumin_interaction'] = df['Bilirubin'] * df['Albumin']
    df['Platelets_Prothrombin_interaction'] = df['Platelets'] * df['Prothrombin']
    
    return df
pp_train = convert_age_and_create_interactions(df_train)
test_df = convert_age_and_create_interactions(df_test)


In [12]:
pp_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7905 entries, 0 to 7904
Data columns (total 27 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 7905 non-null   int64  
 1   N_Days                             7905 non-null   int64  
 2   Drug                               7905 non-null   object 
 3   Age                                7905 non-null   int64  
 4   Sex                                7905 non-null   object 
 5   Ascites                            7905 non-null   object 
 6   Hepatomegaly                       7905 non-null   object 
 7   Spiders                            7905 non-null   object 
 8   Edema                              7905 non-null   object 
 9   Bilirubin                          7905 non-null   float64
 10  Cholesterol                        7905 non-null   float64
 11  Albumin                            7905 non-null   float

In [ ]:
def master_preprocessing(df):
    """
    Apply all preprocessing steps to the dataframe.

    :param df: DataFrame to be processed
    :param features_to_transform: List of features to transform outliers in
    :return: Preprocessed DataFrame
    """
    # Apply preprocessing for new features
    df = convert_age_and_create_interactions(df)
    return df

In [17]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from hyperopt import hp, tpe, Trials, fmin, STATUS_OK, space_eval


# Encoding the target variable 'Status' if it's categorical
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(pp_train['Status'])

# Selecting categorical columns (excluding the target variable 'Status')
cat_cols = pp_train.select_dtypes(include=['object']).drop(columns=['Status']).columns

# Preprocessing with OneHotEncoder and StandardScaler
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), pp_train.select_dtypes(exclude=['object']).drop(['id'], axis=1).columns),
        ('cat', OneHotEncoder(), cat_cols)
    ])

# Fit and transform the training data
X = preprocessor.fit_transform(pp_train.drop(columns=['Status', 'id']))  # Assuming 'id' is not a feature

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the space of hyperparameters to search
space = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300,400,500,600]),
    'max_depth': hp.choice('max_depth', [3, 4, 5, 6, 7, 8, 9, 10]),
    'learning_rate': hp.choice('learning_rate', [0.01, 0.1, 0.2, 0.3]),
    'subsample': hp.uniform('subsample', 0.8, 1),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),
    'base_score': hp.uniform('base_score', 0, 1),
    'booster': hp.choice('booster', ['gbtree', 'gblinear', 'dart'])

    # Add more hyperparameters here
}

def objective(params):
    xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=3, eval_metric='mlogloss', **params)
    xgb_model.fit(X_train, y_train)
    y_pred_class = xgb_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_class)
    return {'loss': -accuracy, 'status': STATUS_OK}

# Run the hyperparameter search using the tpe algorithm
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=150,  # Adjust based on your computation resource
            trials=Trials())

print("Best parameters: ", best)

# Evaluating the best model
best_params = space_eval(space, best)
best_xgb = xgb.XGBClassifier(objective='multi:softprob', num_class=3, eval_metric='mlogloss',n_jobs=-1,**best_params)
best_xgb.fit(X_train, y_train)

# Making predictions and calculating accuracy on the test set
y_pred_class = best_xgb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_class)
print(f"Model Accuracy with best estimator: {accuracy:.2f}")

# Transform the test data using the same preprocessor
X_new = preprocessor.transform(test_df.drop(columns=['id']))

# Making predictions on the new data
predictions = best_xgb.predict_proba(X_new)

# Creating submission file
prediction_columns = ['Status_C', 'Status_CL', 'Status_D']
submission_df = pd.DataFrame(predictions, columns=prediction_columns)
submission_df['id'] = test_df['id']
submission_df = submission_df[['id'] + prediction_columns]
submission_df.to_csv('submission.csv', index=False)


  1%|          | 1/150 [01:45<4:22:27, 105.69s/trial, best loss: -0.8298545224541429]


KeyboardInterrupt: 

In [4]:
# Creating the DataFrame for submission
prediction_columns = ['Status_C', 'Status_CL', 'Status_D']
submission_df = pd.DataFrame(predictions, columns=prediction_columns)
submission_df['id'] = test_df['id']

# Reordering columns to make 'id' the first column
submission_df = submission_df[['id'] + prediction_columns]

# Saving the DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)

,id,Status_C,Status_CL,Status_D
0,7905,0.377510,0.008224,0.614266
1,7906,0.311397,0.470052,0.218551
2,7907,0.028307,0.004294,0.967399
3,7908,0.984989,0.002151,0.012860
4,7909,0.854317,0.023994,0.121689
